# Intro

- pandas library 를 이용해서 시계열 데이터를 다루는 법을 알아보자.
- R, Python 분석과 프로그래밍의 친구 (by R Friend) 의 Python 데이터 전처리, resmple 부분의 예시를 이용하였다.
- 먼저 [년-월-일 시간:분:초] 형태로 되어있는 index 의 데이터이여야 한다. 그러므로 그렇게 되어있는 데이터를 사용하거나, 그렇게 만든 뒤에 index 를 하도록 하자.

In [4]:
import pandas as pd
import numpy as np

# generate time series index
range = pd.date_range('2019-12-19', '2019-12-20', freq='2min')
df = pd.DataFrame(index = range)[:20]

# add 'price' columm using random number
np.random.seed(seed=1004) # for reproducibility
df['price'] = np.random.randint(low=10, high=100, size=20)

# add 'amount' column unsing random number
df['amount'] = np.random.randint(low=1, high=5, size=20)

In [5]:
print('Shape of df DataFrame:', df.shape)
df

Shape of df DataFrame: (20, 2)


,price,amount
2019-12-19 00:00:00,12,4
2019-12-19 00:02:00,21,2
2019-12-19 00:04:00,41,1
2019-12-19 00:06:00,79,4
2019-12-19 00:08:00,61,2
2019-12-19 00:10:00,81,1
2019-12-19 00:12:00,24,3
2019-12-19 00:14:00,62,1
2019-12-19 00:16:00,76,3
2019-12-19 00:18:00,63,1


# Resampling

## 10분 단위 구간별로 각 칼럼의 첫번째 값(first value), 마지막 값(last value) 구하기

- resample('10T') 는 년-월-일 시간:분:초' 의 시계열 index를 10분 단위의 동일 간격별로 데이터를 뽑으라는 뜻
    - 5분 단위 구간    : resample('5T')
    - 10분 단위 구간  : resample('10T')
    - 20분 단위 구간 : resample('20T')
    - 1시간 단위 구간 : resample('1H')
    - 1일 단위 구간    : resample('1D')
    - 1주일 단위 구각 : resample('1W')
    - 1달 단위 구간    : resample('1M')
    - 1년 단위 구간    : resample('1Y')

- resample 자체는 그냥 데이터를 탁탁 끊어주는 역활에 불과하다.
    - 그래서 이름이 resample 인 것.(구조만 살짝 다시 바꾸는거니까)
- resample('10T') 이후에 나오는 함수들에 의해서 그 특성이 결정된다.
- resample('10T') 의 의미는 10분 간격이 되게 처음{시작값:10분 ~ 마지막 값:19분59초}까지를 모아놓은 Ordered SET 들을 많이 만들어 놓겠다는 뜻. 
- 즉 그 이후의 값은 우리가 하기 나름이다. first 를 취하면 10분간격이 되는거고, last 를 취하면 20분보다 작은 값중 제일 20분에 가까운 값을 가지게 된다.

In [32]:
df

,price,amount
2019-12-19 00:00:00,12,4
2019-12-19 00:02:00,21,2
2019-12-19 00:04:00,41,1
2019-12-19 00:06:00,79,4
2019-12-19 00:08:00,61,2
2019-12-19 00:10:00,81,1
2019-12-19 00:12:00,24,3
2019-12-19 00:14:00,62,1
2019-12-19 00:16:00,76,3
2019-12-19 00:18:00,63,1


In [27]:
# 10분간격으로 끊었을 때에 맨 처음 값
# index 의 값 그대로 출력한다.
df['price'].resample('10T').first()

2019-12-19 00:00:00    12
2019-12-19 00:10:00    81
2019-12-19 00:20:00    95
2019-12-19 00:30:00    33
Freq: 10T, Name: price, dtype: int32

In [28]:
# 10분 간격으로 끊었을 때에 맨 마지막 값
# 즉 index 에는 00:00 이라고 되어 있어도, 출력값은 08:00 이다.
df['price'].resample('10T').last()

2019-12-19 00:00:00    61
2019-12-19 00:10:00    63
2019-12-19 00:20:00    30
2019-12-19 00:30:00    96
Freq: 10T, Name: price, dtype: int32

##  10분 단위 구간별로 숫자형 데이터의 합계/누적합 구하기

- 이 경우에는 10T 별로 나누어져있는 값에 대해서 sum 함수를 취하면 된다.

In [36]:
df

,price,amount
2019-12-19 00:00:00,12,4
2019-12-19 00:02:00,21,2
2019-12-19 00:04:00,41,1
2019-12-19 00:06:00,79,4
2019-12-19 00:08:00,61,2
2019-12-19 00:10:00,81,1
2019-12-19 00:12:00,24,3
2019-12-19 00:14:00,62,1
2019-12-19 00:16:00,76,3
2019-12-19 00:18:00,63,1


In [39]:
# 각 인덱스의 값은 10T 구간별로 끊고나서의 SUM 이다.
# 즉 214 라는 합은 00:00 ~ 09:59 의 인덱스를 가지는 data set 의 sum인것
df['price'].resample('10T').sum()

2019-12-19 00:00:00    214
2019-12-19 00:10:00    306
2019-12-19 00:20:00    359
2019-12-19 00:30:00    286
Freq: 10T, Name: price, dtype: int32

- cumsum 은 아래와 같이 주어진 데이터와 같은 크기의 누적합을 만든다.
- 즉 우리가 만든 합계 Series 에다가 cumsum 을 적용한다면 누계를 만들 수 있다는 뜻이다.

In [46]:
np.array([1,2,3,4]).cumsum()

array([ 1,  3,  6, 10], dtype=int32)

In [47]:
df['price'].resample('10T').sum().cumsum()

2019-12-19 00:00:00     214
2019-12-19 00:10:00     520
2019-12-19 00:20:00     879
2019-12-19 00:30:00    1165
Freq: 10T, Name: price, dtype: int32

## 10분 단위 구간별로 최소값, 최대값, 평균, 중앙값, 범위 구하기

In [51]:
df

,price,amount
2019-12-19 00:00:00,12,4
2019-12-19 00:02:00,21,2
2019-12-19 00:04:00,41,1
2019-12-19 00:06:00,79,4
2019-12-19 00:08:00,61,2
2019-12-19 00:10:00,81,1
2019-12-19 00:12:00,24,3
2019-12-19 00:14:00,62,1
2019-12-19 00:16:00,76,3
2019-12-19 00:18:00,63,1


In [49]:
df['price'].resample('10T').min()

2019-12-19 00:00:00    12
2019-12-19 00:10:00    24
2019-12-19 00:20:00    30
2019-12-19 00:30:00    22
Freq: 10T, Name: price, dtype: int32

In [50]:
df['price'].resample('10T').max()

2019-12-19 00:00:00    79
2019-12-19 00:10:00    81
2019-12-19 00:20:00    95
2019-12-19 00:30:00    96
Freq: 10T, Name: price, dtype: int32

In [52]:
df['price'].resample('10T').mean()

2019-12-19 00:00:00    42.8
2019-12-19 00:10:00    61.2
2019-12-19 00:20:00    71.8
2019-12-19 00:30:00    57.2
Freq: 10T, Name: price, dtype: float64

In [53]:
df['price'].resample('10T').median()

2019-12-19 00:00:00    41
2019-12-19 00:10:00    63
2019-12-19 00:20:00    82
2019-12-19 00:30:00    58
Freq: 10T, Name: price, dtype: int32

In [57]:
df['price'].resample('10T').max() - df['price'].resample('10T').min()

2019-12-19 00:00:00    67
2019-12-19 00:10:00    57
2019-12-19 00:20:00    65
2019-12-19 00:30:00    74
Freq: 10T, Name: price, dtype: int32

## 10분 단위 구간별로 분산, 표준편차 구하기

In [60]:
df

,price,amount
2019-12-19 00:00:00,12,4
2019-12-19 00:02:00,21,2
2019-12-19 00:04:00,41,1
2019-12-19 00:06:00,79,4
2019-12-19 00:08:00,61,2
2019-12-19 00:10:00,81,1
2019-12-19 00:12:00,24,3
2019-12-19 00:14:00,62,1
2019-12-19 00:16:00,76,3
2019-12-19 00:18:00,63,1


In [61]:
df['price'].resample('10T').var()

2019-12-19 00:00:00    767.2
2019-12-19 00:10:00    499.7
2019-12-19 00:20:00    624.2
2019-12-19 00:30:00    930.7
Freq: 10T, Name: price, dtype: float64

In [62]:
# 표준편차는 그냥 sqrt 를 취하면 된다.
np.sqrt(df['price'].resample('10T').var())

2019-12-19 00:00:00    27.698375
2019-12-19 00:10:00    22.353971
2019-12-19 00:20:00    24.983995
2019-12-19 00:30:00    30.507376
Freq: 10T, Name: price, dtype: float64

## 10분 단위별로 몇개의 sample 이 있는지 구하기

- df 데이터의 경우 10분 간격으로 얌전히 끊겨있어서 별 좋은 정보는 아니다.
- 하지만 시계열이지만, 시간 간격이 중구난방으로 되어있는경우 이런 조치가 필요하다.

In [64]:
df['price'].resample('10T').count()

2019-12-19 00:00:00    5
2019-12-19 00:10:00    5
2019-12-19 00:20:00    5
2019-12-19 00:30:00    5
Freq: 10T, Name: price, dtype: int64

# pd.date_range

- Pandas 에서는 data 의 빈도와 날짜 빈도를 생성할 수 있다.

In [67]:
# 3일 주기의 데이터를 생성하기
pd.date_range('2019-12-01', periods = 8, freq = '3d')

DatetimeIndex(['2019-12-01', '2019-12-04', '2019-12-07', '2019-12-10',
               '2019-12-13', '2019-12-16', '2019-12-19', '2019-12-22'],
              dtype='datetime64[ns]', freq='3D')

In [66]:
# 
pd.date_range('2019-12-01', periods = 8, freq = 3 * '1d') 

DatetimeIndex(['2019-12-01', '2019-12-04', '2019-12-07', '2019-12-10',
               '2019-12-13', '2019-12-16', '2019-12-19', '2019-12-22'],
              dtype='datetime64[ns]', freq='3D')

In [68]:
pd.date_range('2020-01-01', periods=10, freq='d')

DatetimeIndex(['2020-01-01', '2020-01-02', '2020-01-03', '2020-01-04',
               '2020-01-05', '2020-01-06', '2020-01-07', '2020-01-08',
               '2020-01-09', '2020-01-10'],
              dtype='datetime64[ns]', freq='D')

# 이동 평균